# 02. Exploratory Data Analysis (EDA)

## Objective
- Phân tích hành vi tiêu thụ điện
- Phát hiện các mô hình thời vụ

## 1. Setup and Load Dataset

In [ ]:
from src.data_loader import load_raw_data

df = load_raw_data()
print("✅ Loaded")
print("Shape:", df.shape)

df.head()


## 2. Time Series Visualization
- Chuỗi thời gian tổng
- Chuỗi theo consumer (mẫu đại diện)

In [ ]:
# chuỗi thời gian theo tổng 
df_total = df.sum(axis=1)
plt.figure(figsize=(15, 5))
plt.plot(df_total, linewidth=0.8)
plt.title("Total Electricity Load (All Consumers)")
plt.xlabel("Time")
plt.ylabel("Total Consumption (kW)")
plt.grid(alpha=0.3)
plt.show()

## 3. Seasonality Analysis
### 3.1 Daily Seasonality
- Biểu đồ trung bình theo giờ

### 3.2 Weekly Seasonality
- Ngày trong tuần vs cuối tuần

### 3.3 Yearly Seasonality
- Biến động theo tháng / mùa

## 4. Load Distribution
- Histogram
- Boxplot

## 5. Statistical Summary
- Mean, Median, Std
- Min / Max

## 6. Key Insights
- Các mẫu thời vụ quan trọng
- Nhận dạng giờ cao điểm / thấp điểm

## 7. Conclusion
- Tóm tắt insight chính phục vụ modeling